### Load a dataset into a dictionary and organize each year as a nested dictionary

<pre>
key = country name
key = year
values = [fixed, mobile, internet users, rate of growth x 3 (fixed / mobile / internet users)]
</pre>

In [17]:
import csv
import pycountry
import pycountry_convert

# filename - dataset to load
# dataColumn - column of dataset to convert to float
# position - position within the list to place value
def loadDataset(continents, countries, years, filename, dataColumn, position):
    f = open(filename)
    newCountries = 0
    totValues = 0
    for row in csv.reader(f, delimiter=','):
        try:
            key = row[0]
            
            # lookup country
            country = pycountry.countries.lookup(key)                         
            name = country.name
            
            # lookup continent
            cn_continent = pycountry_convert.convert_country_alpha2_to_continent(country.alpha_2)
            
            if not cn_continent in continents:
                continents[cn_continent] = set()
            
            continents[cn_continent].add(country.name) 
                        
            year = int(row[1])
            value = float(row[dataColumn])
            
            # exclude very old entries or entries with zero values 
            if (value > 0 and year>=1990):            
                if not (name in countries) :
                    # first encounter of this country
                    newCountries += 1
                    countries[name] = {}          
            
                if not (year in countries[row[0]]):        
                    # first encounter of this year
                    # construct list with missing values for this year
                    countries[name][year] = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

                countries[name][year][position] = value

                totValues+=1

                # keep min/max years
                if (year < years['min']):
                    years['min'] = year

                if (year > years['max']):
                    years['max'] = year        

        except (KeyError):
            pass
                    
        except (IndexError):
            pass

        except (ValueError):
            pass
            
        except (LookupError):
            pass

    print("Encountered", newCountries, "new countries. Loaded", totValues, "values - Year min", years['min'], "max", years['max'])

Identify missing entries for all countries given min/max years - add missing values (with zero)

In [7]:
def missingEntries(countries, years):
    totMissing = 0
    misValues = {}
    minYear = years['min']
    maxYear = years['max']

    #identify missing years
    for country, data in countries.items():        
        for year in range(minYear, maxYear + 1):
            if not year in data:
                if not country in misValues:
                    misValues[country] = []

                misValues[country].append(year)
                totMissing += 1

    # fill in missing years
    for country, data in misValues.items():        
        for year in data:
            countries[country][year] = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

    print(totMissing, "missing years in", len(misValues), "countries")

Fill in missing values by averaging out previous/next available values

In [8]:
def fillMissing(countries, years):
    totMissing = 0
    totCorrected = 0
    for name, value in countries.items():
        # iterate through 3 datasets
        for dataset in range(0,3):
            prevValue = 0
            year = years['min']
            maxYear = years['max']
            while (year < maxYear):
                value = countries[name][year][dataset]

                if (value == 0 and prevValue == 0):
                    # Value is missing but we are at the start of the series, can't do nothing        
                    pass

                elif (value > 0):
                    prevValue = value

                else :                
                    totMissing += 1

                    # identify next value
                    nextYear = year + 1
                    while (nextYear < maxYear):
                        nextValue = countries[name][nextYear][dataset]

                        if (nextValue > 0):
                            period = nextYear - year + 1                                       
                            increase = (nextValue - prevValue) / period                    

                            # average out missing values
                            for slot in range(year, nextYear):
                                missingValue = prevValue + increase                        
                                countries[name][slot][dataset] = missingValue
                                prevValue = missingValue
                                totCorrected += 1

                            break

                        totMissing += 1
                        nextYear += 1

                year += 1

    print(totMissing, "missing values -", totCorrected, "corrected")

Load actual data - all 3 datasets

In [18]:
countries = {}
continents = {}
years = {}
years['min'] = 3000
years['max'] = 0

# Start by loading fixed internet broadband subscriptions per 100 inhabitants 
loadDataset(continents, countries, years, "../dataset/Fixed_Internet_broadband_Subscriptions_per_100_inhabitants_20170325_164431694_export.csv", 3, 0)

# Load 2nd dataset with mobile cellular telephony subscriptions per 100 inhabitants 
loadDataset(continents, countries, years, "../dataset/Mobile-cellular_telephone_subscriptions_per_100_inhabitants_20170325_164502263.csv", 2, 1)

# Load 3rd data set with proportion of individuals using the Internet per 100 inhabitants
loadDataset(continents, countries, years, "../dataset/Proportion_of_individuals_using_the_Internet_20170325_164406087_export.csv", 3, 2)

# identify missing entries and fill in with zeros
missingEntries(countries, years)

# fill in missing values
fillMissing(countries, years)

Encountered 197 new countries. Loaded 2368 values - Year min 2000 max 2015
Encountered 8 new countries. Loaded 3891 values - Year min 1990 max 2015
Encountered 4 new countries. Loaded 3002 values - Year min 1990 max 2015
1083 missing years in 154 countries
706 missing values - 170 corrected


In [11]:
countries['Greece']

{1990: [0.0, 0.0, 0.0],
 1991: [0.0, 0.0, 0.0],
 1992: [0.0, 0.0, 0.0],
 1993: [0.0, 0.458787632652948, 0.0, 0.0, 0.0, 0.0],
 1994: [0.0, 1.44715350108522, 0.0, 0.0, 0.0, 0.0],
 1995: [0.0, 2.55814173585753, 0.0, 0.0, 0.0, 0.0],
 1996: [0.0, 4.94498793869107, 0.0, 0.0, 0.0, 0.0],
 1997: [0.0, 8.6554536834923, 0.0, 0.0, 0.0, 0.0],
 1998: [0.0, 18.7853952591242, 0.0, 0.0, 0.0, 0.0],
 1999: [0.0, 35.660380184959, 0.0, 0.0, 0.0, 0.0],
 2000: [0.0, 53.9953233323773, 9.138837308, 0.0, 0.0, 0.0],
 2001: [0.0, 72.3126477410963, 10.93502581, 0.0, 0.0, 0.0],
 2002: [0.0, 84.4726790115104, 14.67, 0.0, 0.0, 0.0],
 2003: [0.094962457, 81.0045534117592, 17.8, 0.0, 0.0, 0.0],
 2004: [0.466276319, 84.4955126737809, 21.42, 0.0, 0.0, 0.0],
 2005: [1.450083239, 92.9245487220107, 24.0, 0.0, 0.0, 0.0],
 2006: [4.416913654, 99.3423396704135, 32.25, 0.0, 0.0, 0.0],
 2007: [9.193870651, 111.096420636559, 35.88, 0.0, 0.0, 0.0],
 2008: [13.59384134, 124.508360188876, 38.2, 0.0, 0.0, 0.0],
 2009: [17.25518372, 1

In [19]:
continents

{'Africa': {'Algeria',
  'Angola',
  'Benin',
  'Botswana',
  'Burkina Faso',
  'Burundi',
  'Cabo Verde',
  'Cameroon',
  'Central African Republic',
  'Chad',
  'Comoros',
  'Congo',
  "Côte d'Ivoire",
  'Djibouti',
  'Egypt',
  'Equatorial Guinea',
  'Eritrea',
  'Ethiopia',
  'Gabon',
  'Gambia',
  'Ghana',
  'Guinea',
  'Guinea-Bissau',
  'Kenya',
  'Lesotho',
  'Liberia',
  'Libya',
  'Madagascar',
  'Malawi',
  'Mali',
  'Mauritania',
  'Mauritius',
  'Mayotte',
  'Morocco',
  'Mozambique',
  'Namibia',
  'Niger',
  'Nigeria',
  'Rwanda',
  'Sao Tome and Principe',
  'Senegal',
  'Seychelles',
  'Sierra Leone',
  'Somalia',
  'South Africa',
  'South Sudan',
  'Sudan',
  'Swaziland',
  'Tanzania, United Republic of',
  'Togo',
  'Tunisia',
  'Uganda',
  'Zambia',
  'Zimbabwe'},
 'Asia': {'Afghanistan',
  'Armenia',
  'Azerbaijan',
  'Bahrain',
  'Bangladesh',
  'Bhutan',
  'Brunei Darussalam',
  'Cambodia',
  'China',
  'Cyprus',
  'Georgia',
  'India',
  'Indonesia',
  'Iraq',
